Race Data from 1990 to 1999

In [2]:
import pandas as pd

# Read the file
with open('Race/race_1990_2000.txt', 'r') as file:
    lines = file.readlines()

In [3]:
data = []
for line in lines:
    if line.strip() and line[:4].isdigit():  # Skip empty lines and non-data lines
        l = line.strip().split()
        year = l[0]
        fips_code = l[1]
        white = l[2]
        black = l[3]
        native_american = l[4]
        asian_pacific_islander = l[5]
        hispanic_white = l[6]
        hispanic_black = l[7]
        hispanic_native_american = l[8]
        hispanic_asian_pacific_islander = l[9]
        
        data.append([year, fips_code, white, black, native_american, asian_pacific_islander,
                     hispanic_white, hispanic_black, hispanic_native_american, hispanic_asian_pacific_islander])

In [4]:
# Convert the data to a DataFrame
columns = ['Year', 'FIPS Code', 'White', 'Black', 'Native American', 'Asian/Pacific Islander', 
           'Hispanic White', 'Hispanic Black', 'Hispanic Native American', 'Hispanic Asian/Pacific Islander']
df = pd.DataFrame(data, columns=columns)
df

,Year,FIPS Code,White,Black,Native American,Asian/Pacific Islander,Hispanic White,Hispanic Black,Hispanic Native American,Hispanic Asian/Pacific Islander
0,1990,01001,27085,6854,71,118,184,41,0,3
1,1991,01001,27560,7016,74,124,196,46,0,2
2,1992,01001,28257,7275,75,133,210,48,0,4
3,1993,01001,28942,7538,71,140,234,47,0,4
4,1994,01001,29819,7853,71,150,266,50,0,9
...,...,...,...,...,...,...,...,...,...,...
31405,1995,56045,6368,4,86,12,93,1,2,0
31406,1996,56045,6385,4,88,11,92,2,1,0
31407,1997,56045,6321,5,81,12,88,5,1,1
31408,1998,56045,6295,4,90,11,95,3,1,1


In [5]:
# Create a FIPS to state mapping dictionary
fips_to_state = {
    '01': 'Alabama',
    '02': 'Alaska',
    '04': 'Arizona',
    '05': 'Arkansas',
    '06': 'California',
    '08': 'Colorado',
    '09': 'Connecticut',
    '10': 'Delaware',
    '12': 'Florida',
    '13': 'Georgia',
    '15': 'Hawaii',
    '16': 'Idaho',
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '20': 'Kansas',
    '21': 'Kentucky',
    '22': 'Louisiana',
    '23': 'Maine',
    '24': 'Maryland',
    '25': 'Massachusetts',
    '26': 'Michigan',
    '27': 'Minnesota',
    '28': 'Mississippi',
    '29': 'Missouri',
    '30': 'Montana',
    '31': 'Nebraska',
    '32': 'Nevada',
    '33': 'New Hampshire',
    '34': 'New Jersey',
    '35': 'New Mexico',
    '36': 'New York',
    '37': 'North Carolina',
    '38': 'North Dakota',
    '39': 'Ohio',
    '40': 'Oklahoma',
    '41': 'Oregon',
    '42': 'Pennsylvania',
    '44': 'Rhode Island',
    '45': 'South Carolina',
    '46': 'South Dakota',
    '47': 'Tennessee',
    '48': 'Texas',
    '49': 'Utah',
    '50': 'Vermont',
    '51': 'Virginia',
    '53': 'Washington',
    '54': 'West Virginia',
    '55': 'Wisconsin',
    '56': 'Wyoming'
}

# Ensure the FIPS code is in the correct format (two digits)
df['State FIPS'] = df['FIPS Code'].astype(str).str[:2]

# Map FIPS code to state name
df['State'] = df['State FIPS'].map(fips_to_state)
df['State & Year'] = df['State'] + ', ' + df['Year']

# Convert race columns to numeric, if they are not already
race_columns = ['White', 'Black', 'Native American', 'Asian/Pacific Islander', 
                'Hispanic White', 'Hispanic Black', 'Hispanic Native American', 'Hispanic Asian/Pacific Islander']
df[race_columns] = df[race_columns].apply(pd.to_numeric, errors='coerce')

# Aggregate by state
aggregated_data = df.groupby('State & Year')[race_columns].sum().reset_index()

# Display the aggregated DataFrame
print(aggregated_data.head())



    State & Year    White    Black  Native American  Asian/Pacific Islander  \
0  Alabama, 1990  2966064  1019884            16249                   21516   
1  Alabama, 1991  2993899  1033240            16214                   22350   
2  Alabama, 1992  3023386  1050520            15751                   23149   
3  Alabama, 1993  3056573  1068400            15482                   24370   
4  Alabama, 1994  3078356  1083627            15153                   25156   

   Hispanic White  Hispanic Black  Hispanic Native American  \
0           20164            3583                       351   
1           20737            3510                       355   
2           21812            3541                       366   
3           23486            3627                       384   
4           25658            3741                       420   

   Hispanic Asian/Pacific Islander  
0                              697  
1                              720  
2                              744 

In [70]:
aggregated_data.to_excel('race_data_1990_1999.xlsx', index=False)

Race Data from 2000 to 2010

In [5]:
import pandas as pd

# Load the data
file_path = 'st-est00int-sexracehisp (1).csv'
data = pd.read_csv(file_path)

# Melt the dataframe to combine years into a single column
melted_data = pd.melt(data, id_vars=['NAME', 'RACE', 'ORIGIN'], var_name='Year', value_name='Population')

# Create 'State & Year' column
melted_data['State & Year'] = melted_data['NAME'] + ', ' + melted_data['Year']

# Define a function to map ORIGIN to readable categories
def map_origin(origin):
    if origin == 1:
        return 'Not Hispanic'
    elif origin == 2:
        return 'Hispanic'

melted_data['Origin Category'] = melted_data['ORIGIN'].apply(map_origin)

# Combine RACE and ORIGIN into a single category
def combine_race_origin(race, origin):
    race_map = {
        0: 'Unknown/Unreported',
        1: 'White',
        2: 'Black or African American',
        3: 'American Indian or Alaska Native',
        4: 'Asian',
        5: 'Native Hawaiian or Other Pacific Islander',
        6: 'Others'
    }
    return f"{origin} {race_map[race]}"

melted_data['Race & Origin'] = melted_data.apply(lambda row: combine_race_origin(row['RACE'], row['Origin Category']), axis=1)

# Pivot the table to have combined race and origin as columns
pivot_data = melted_data.pivot_table(index=['State & Year'], columns='Race & Origin', values='Population').reset_index()
pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].astype(int)

# Display the final dataframe to the user
#import ace_tools as tools; tools.display_dataframe_to_user(name="Final Data with State, Year, Race, and Origin", dataframe=pivot_data)

pivot_data


Race & Origin,State & Year,Hispanic American Indian or Alaska Native,Hispanic Asian,Hispanic Black or African American,Hispanic Native Hawaiian or Other Pacific Islander,Hispanic Others,Hispanic Unknown/Unreported,Hispanic White,Not Hispanic American Indian or Alaska Native,Not Hispanic Asian,Not Hispanic Black or African American,Not Hispanic Native Hawaiian or Other Pacific Islander,Not Hispanic Others,Not Hispanic Unknown/Unreported,Not Hispanic White
0,"Alabama, 2000",1306.0,566.0,7345.0,525.0,1427.0,77016.0,65847.0,21956.0,31978.0,1154109.0,1083.0,35003.0,4375157.0,3131028.0
1,"Alabama, 2001",1801.0,681.0,7654.0,802.0,1836.0,85722.0,72948.0,22283.0,33290.0,1159749.0,1161.0,36990.0,4381912.0,3128439.0
2,"Alabama, 2002",2317.0,744.0,8018.0,1039.0,2233.0,94179.0,79828.0,22649.0,35066.0,1163654.0,1233.0,38857.0,4385910.0,3124451.0
3,"Alabama, 2003",2818.0,844.0,8408.0,1299.0,2720.0,103472.0,87383.0,23102.0,37085.0,1169990.0,1326.0,40820.0,4400019.0,3127696.0
4,"Alabama, 2004",3307.0,979.0,8838.0,1559.0,3094.0,113284.0,95507.0,23448.0,39733.0,1177537.0,1424.0,43038.0,4417445.0,3132265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,"Wyoming, 2006",1904.0,255.0,611.0,118.0,1304.0,40433.0,36241.0,11160.0,3623.0,4060.0,363.0,6662.0,482234.0,456366.0
557,"Wyoming, 2007",2172.0,263.0,662.0,122.0,1408.0,43022.0,38395.0,11390.0,3767.0,4283.0,373.0,7126.0,491854.0,464915.0
558,"Wyoming, 2008",2379.0,288.0,663.0,137.0,1545.0,45949.0,40937.0,11587.0,3967.0,4251.0,366.0,7474.0,500094.0,472449.0
559,"Wyoming, 2009",2465.0,322.0,787.0,140.0,1638.0,48769.0,43417.0,11839.0,4274.0,4615.0,363.0,7933.0,511082.0,482058.0


In [6]:
pivot_data.to_excel('race_data_2000_2010.xlsx', index=False)

Race Data from 2011 to 2020

In [9]:
import pandas as pd

# Load the data
file_path = 'st-est00int-sexracehisp (2).csv'
data = pd.read_csv(file_path)


# Melt the dataframe to combine years into a single column
melted_data = pd.melt(data, id_vars=['NAME', 'RACE', 'ORIGIN'], var_name='Year', value_name='Population')

# Create 'State & Year' column
melted_data['State & Year'] = melted_data['NAME'] + ', ' + melted_data['Year']

# Define a function to map ORIGIN to readable categories
def map_origin(origin):
    if origin == 1:
        return 'Not Hispanic'
    elif origin == 2:
        return 'Hispanic'

melted_data['Origin Category'] = melted_data['ORIGIN'].apply(map_origin)

# Combine RACE and ORIGIN into a single category
def combine_race_origin(race, origin):
    race_map = {
        1: 'White',
        2: 'Black or African American',
        3: 'American Indian or Alaska Native',
        4: 'Asian',
        5: 'Native Hawaiian or Other Pacific Islander',
        6: 'Others'
    }
    return f"{origin} {race_map[race]}"

melted_data['Race & Origin'] = melted_data.apply(lambda row: combine_race_origin(row['RACE'], row['Origin Category']), axis=1)

# Sum population by 'State & Year' and 'Race & Origin'
summed_data = melted_data.groupby(['State & Year', 'Race & Origin'])['Population'].sum().reset_index()

# Pivot the table to have combined race and origin as columns
pivot_data = summed_data.pivot_table(index=['State & Year'], columns='Race & Origin', values='Population').reset_index()

pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].astype(int)


pivot_data


Race & Origin,State & Year,Hispanic American Indian or Alaska Native,Hispanic Asian,Hispanic Black or African American,Hispanic Native Hawaiian or Other Pacific Islander,Hispanic Others,Hispanic White,Not Hispanic American Indian or Alaska Native,Not Hispanic Asian,Not Hispanic Black or African American,Not Hispanic Native Hawaiian or Other Pacific Islander,Not Hispanic Others,Not Hispanic White
0,"Alabama, 2011",6581.0,1605.0,13125.0,2984.0,6141.0,158892.0,26147.0,56036.0,1254207.0,2049.0,61297.0,3210578.0
1,"Alabama, 2012",6555.0,1460.0,13532.0,2813.0,6218.0,159311.0,26225.0,59533.0,1262757.0,2172.0,63962.0,3212094.0
2,"Alabama, 2013",6418.0,1464.0,13839.0,2666.0,6448.0,161253.0,26358.0,61841.0,1271477.0,2257.0,66154.0,3211411.0
3,"Alabama, 2014",6371.0,1488.0,14067.0,2647.0,6448.0,163339.0,26553.0,63841.0,1277797.0,2299.0,68293.0,3210594.0
4,"Alabama, 2015",6426.0,1619.0,14476.0,2609.0,6590.0,166401.0,26694.0,66017.0,1284438.0,2375.0,70490.0,3206668.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,"Wyoming, 2017",4028.0,467.0,968.0,176.0,2231.0,50376.0,12070.0,5742.0,6204.0,404.0,9794.0,487534.0
557,"Wyoming, 2018",4151.0,489.0,1033.0,177.0,2374.0,50517.0,12134.0,5982.0,6100.0,398.0,10014.0,485685.0
558,"Wyoming, 2019",4186.0,498.0,1044.0,189.0,2519.0,51079.0,12076.0,5923.0,6070.0,409.0,10350.0,485773.0
559,"Wyoming, 2020",4185.0,525.0,1055.0,209.0,2574.0,51731.0,12189.0,5869.0,6028.0,408.0,10767.0,486788.0


In [10]:
pivot_data.to_excel('race_data_2011_2020.xlsx', index=False)

Race Data from 2021 to 2023

In [11]:
import pandas as pd

# Load the data
file_path = 'st-est00int-sexracehisp (3).csv'
data = pd.read_csv(file_path)


# Melt the dataframe to combine years into a single column
melted_data = pd.melt(data, id_vars=['NAME', 'RACE', 'ORIGIN'], var_name='Year', value_name='Population')

# Create 'State & Year' column
melted_data['State & Year'] = melted_data['NAME'] + ', ' + melted_data['Year']

# Define a function to map ORIGIN to readable categories
def map_origin(origin):
    if origin == 1:
        return 'Not Hispanic'
    elif origin == 2:
        return 'Hispanic'

melted_data['Origin Category'] = melted_data['ORIGIN'].apply(map_origin)

# Combine RACE and ORIGIN into a single category
def combine_race_origin(race, origin):
    race_map = {
        1: 'White',
        2: 'Black or African American',
        3: 'American Indian or Alaska Native',
        4: 'Asian',
        5: 'Native Hawaiian or Other Pacific Islander',
    }
    return f"{origin} {race_map[race]}"

melted_data['Race & Origin'] = melted_data.apply(lambda row: combine_race_origin(row['RACE'], row['Origin Category']), axis=1)

# Sum population by 'State & Year' and 'Race & Origin'
summed_data = melted_data.groupby(['State & Year', 'Race & Origin'])['Population'].sum().reset_index()

# Pivot the table to have combined race and origin as columns
pivot_data = summed_data.pivot_table(index=['State & Year'], columns='Race & Origin', values='Population').reset_index()

pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].astype(int)


pivot_data


Race & Origin,State & Year,Hispanic American Indian or Alaska Native,Hispanic Asian,Hispanic Black or African American,Hispanic Native Hawaiian or Other Pacific Islander,Hispanic White,Not Hispanic American Indian or Alaska Native,Not Hispanic Asian,Not Hispanic Black or African American,Not Hispanic Native Hawaiian or Other Pacific Islander,Not Hispanic White
0,"Alabama, 2021",13088.0,4216.0,23724.0,4170.0,237537.0,59452.0,95919.0,1376494.0,5633.0,3331361.0
1,"Alabama, 2022",13850.0,4254.0,24421.0,4209.0,244915.0,60347.0,98411.0,1381588.0,5855.0,3341234.0
2,"Alabama, 2023",14705.0,4434.0,25238.0,4282.0,253281.0,61385.0,100982.0,1388496.0,6176.0,3358909.0
3,"Alabama, AGE",3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0
4,"Alaska, 2021",8904.0,3039.0,4851.0,1108.0,40946.0,141667.0,63092.0,35649.0,15838.0,484128.0
...,...,...,...,...,...,...,...,...,...,...,...
199,"Wisconsin, AGE",3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0,3655.0
200,"Wyoming, 2021",5834.0,979.0,1937.0,383.0,54440.0,16826.0,8899.0,9333.0,1113.0,494376.0
201,"Wyoming, 2022",5921.0,1012.0,1974.0,375.0,55494.0,16829.0,9151.0,9507.0,1158.0,495062.0
202,"Wyoming, 2023",5996.0,1026.0,2030.0,394.0,56352.0,16936.0,9472.0,9513.0,1211.0,496234.0


In [12]:
pivot_data.to_excel('race_data_2021_2023.xlsx', index=False)

In [3]:
import pandas as pd

df = pd.read_excel("Religious Affiliation.xlsx")
df['State'] = df['State'] + ', 2014'
df.to_excel('religious_affiliation_data_2014.xlsx', index=False)